# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770422469602                   -0.52    9.0    194ms
  2   -2.771692635745       -2.90       -1.32    1.0    100ms
  3   -2.771714318464       -4.66       -2.44    1.0   99.4ms
  4   -2.771714634854       -6.50       -3.13    1.0    104ms
  5   -2.771714714684       -7.10       -4.12    2.0    118ms
  6   -2.771714715143       -9.34       -4.58    1.0    106ms
  7   -2.771714715246       -9.99       -5.76    1.0    105ms
  8   -2.771714715249      -11.51       -5.92    2.0    124ms
  9   -2.771714715250      -12.80       -7.27    1.0    109ms
 10   -2.771714715250      -13.74       -7.71    2.0    200ms
 11   -2.771714715250      -14.07       -8.60    1.0    793ms


-0.00013457373784660242

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770405956160                   -0.53   10.0    926ms
  2   -2.771772908908       -2.86       -1.30    1.0    190ms
  3   -2.771801655460       -4.54       -2.65    1.0    147ms
  4   -2.771802044900       -6.41       -3.75    1.0   98.8ms
  5   -2.771802073962       -7.54       -4.16    2.0    135ms
  6   -2.771802074462       -9.30       -5.24    1.0    101ms
  7   -2.771802074476      -10.86       -5.72    2.0    121ms
  8   -2.771802074476      -12.48       -6.24    1.0    104ms
  9   -2.771802074476      -13.61       -7.51    1.0    109ms
 10   -2.771802074476   +  -14.88       -7.54    2.0    124ms
 11   -2.771802074476   +  -14.88       -7.70    1.0    113ms
 12   -2.771802074476   +    -Inf       -8.57    1.0    113ms


0.017612221216980757

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457373784660242
Displaced dipole:  0.017612221216980757
Polarizability :   1.7746794954827358


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  61.1ms  Non-interacting
   1        0       1       -0.60     10.0   1.08s  
   2        0       2       -2.42      8.0   121ms  
   3        0       3       -3.54      5.0   101ms  
   4        0       4       -5.33      4.0  95.0ms  
   5        0       5       -7.24      1.0  87.0ms  
   6        0       6      -10.26      1.0  86.8ms  
   7        1       1       -7.35     13.0   216ms  Restart
   8        1       2       -8.88      1.0  86.2ms  
                                      13.0   118ms  Final orbitals


(δψ = Matrix{ComplexF64}[[-0.0032750387375729685 + 0.00037018510611588895im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.03216775424695795 + 0.30771972820299537im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; 0.004447783591521686 + 0.04082357241426837im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.009398741132250737 - 0.08039064368658297im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206939303904963e-7 -2.502015182483701e-7 … -4.880641150883981e-8 -2.5020143646398996e-7; -6.293616315608888e-7 -4.793036444452175e-7 … -1.207784377767573e-7 -4.79303587440305e-7; … ; 1.362626324626707e-7 1.1356231318965117e-7 … 4.711992949543769e-8 1.1356232758058185e-7; 1.3305478358116244e-7 1.363263906453565e-7 … 5.2006881654141915e-8 1.3632645278298923e-7;;; -2.5020151565836726e-7 -1.7404911224535425e-7 … -3.556493570586156e-8 -1.7404905276658523e-7; -4.793036390081426e-7 -3.679732234307164e-7 … -1.1202895147756253e-7 -3.679731802685683e-7; … ; 1.135623078222947e-7 1.1211098139792225e-7 … 1.0980685782173364e-7 1.12

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125373964221
Interacting polarizability:     1.773654866680391


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 8.97e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.11e-11
┌ Info: GMRES linsolve converged at iteration 2, step 2:
│ * norm of residual = 4.18e-12
└ * number of operations = 11
Non-interacting polarizability: 1.9257125373963317
Interacting polarizability:     1.773654863143447


We obtain the identical result to above.